# **EQNmix**

### EQNmix is a mixed architecture that combines two widely-used neural networks in seismology: ConvNetQuake (Perol et al., 2018) and EQTransformer (Mousavi et al., 2020). Our algorithm employs a Gaussian mixture model for Bayesian Inference using the outputs generated by both neural networks. The ultimate outcome is a probabilistic location pinpointed using just a single seismic station.ks.

##### An integral facet of its versatile design is the algorithm's adaptability, as it is not confined to a single travel-time algorithm. It accommodates a spectrum of options ranging from simpler to more intricate travel-time methods. Furthermore, various sampling techniques such as variational inference, Hamiltonian sampling, among others, can be seamlessly integrated. 
##### This algorithm is applicable not only to individual seismic stations but can also be extended to entire seismic networks.

###### Information of the TEST events obtained by the **Southern California Earthquake Data Center (SCEDC)**

##### **Event A** \| 2019/07/04 19\:21:32.09 eq  l 4.50 w   35.67150 -117.47883   5.2 A 38443871  120 3331
###### Mixing coefficients by CNQ: \[0.06642566, 0.13303314, 0.018152032, 0.2676338, 0.03821565, 0.27928686\]
           
##### **Event B** \| 2019/07/05 12\:38:30.02 eq  l 4.09 w   35.77167 -117.57067   6.8 A 38451079  107 3341
###### Mixing coefficients by CNQ: \[0.06656365, 0.13407934, 0.018142378, 0.2691841, 0.038651247, 0.2805622\]

##### **Event C** \| 2019/07/06 23\:50:41.99 eq  l 4.50 w   35.82350 -117.66300   6.5 A 38469375  210 2460
###### Mixing coefficients by CNQ: \[0.0666608, 0.13491394, 0.018160287, 0.27093622, 0.03867901, 0.28238913\]

In [1]:
# Importing libraries
import numpy as np
import pymc3 as pm

### TEST EVENT A

In [56]:
outeqt='/Users/jorge/EQTransformer/examples/detectionsCLC/CLC_outputs/X_prediction_results.csv'


In [57]:
import pandas as pd

In [58]:
df=pd.read_csv(outeqt)

In [59]:
df_f = df[(df['detection_probability'] > 0.95) & 
                 (df['s_probability'] > 0.88) & 
                 (df['p_probability'] > 0.8)]

# Imprimir el DataFrame filtrado


In [60]:
df_f['p_arrival_time'] = pd.to_datetime(df_f['p_arrival_time']).apply(UTCDateTime)
df_f['s_arrival_time'] = pd.to_datetime(df_f['s_arrival_time']).apply(UTCDateTime)

# Calcular la diferencia entre s_arrival_time y p_arrival_time
df_f['t_observed'] = df_f['s_arrival_time'] - df_f['p_arrival_time']


/var/folders/8b/qn3hfzzd03x7b9_df_n_71zc0000gn/T/ipykernel_37722/929892283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['p_arrival_time'] = pd.to_datetime(df_f['p_arrival_time']).apply(UTCDateTime)
/var/folders/8b/qn3hfzzd03x7b9_df_n_71zc0000gn/T/ipykernel_37722/929892283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['s_arrival_time'] = pd.to_datetime(df_f['s_arrival_time']).apply(UTCDateTime)
/var/folders/8b/qn3hfzzd03x7b9_df_n_71zc0000gn/T/ipykernel_37722/929892283.py:5: SettingWith

In [72]:
t_obs=df_f['t_observed'].iloc[0]

In [74]:
df_f

,file_name,network,station,instrument_type,station_lat,station_lon,station_elv,event_start_time,event_end_time,detection_probability,detection_uncertainty,p_arrival_time,p_probability,p_uncertainty,p_snr,s_arrival_time,s_probability,s_uncertainty,s_snr,t_observed
2430,CLC_CI_HH_2019-07-05T13:48:48.008300Z,CI,CLC,HH,35.81574,-117.59751,775,2019-07-05 13:49:42.648300,2019-07-05 13:49:45.238300,0.99,NaN,2019-07-05T13:49:42.618300Z,0.82,NaN,10.2,2019-07-05T13:49:43.468300Z,0.89,NaN,6.4,0.85
2454,CLC_CI_HH_2019-07-05T14:01:24.008300Z,CI,CLC,HH,35.81574,-117.59751,775,2019-07-05 14:01:48.688300,2019-07-05 14:01:50.498300,0.99,NaN,2019-07-05T14:01:48.678300Z,0.84,NaN,31.7,2019-07-05T14:01:49.448300Z,0.89,NaN,11.1,0.77
3740,CLC_CI_HH_2019-07-06T01:14:54.008300Z,CI,CLC,HH,35.81574,-117.59751,775,2019-07-06 01:15:22.528300,2019-07-06 01:15:24.568300,0.98,NaN,2019-07-06T01:15:22.528300Z,0.83,NaN,17.4,2019-07-06T01:15:23.158300Z,0.89,NaN,5.2,0.63
6035,CLC_CI_HH_2019-07-06T21:26:36.008300Z,CI,CLC,HH,35.81574,-117.59751,775,2019-07-06 21:27:28.128300,2019-07-06 21:27:30.888300,0.99,NaN,2019-07-06T21:27:28.108300Z,0.85,NaN,25.5,2019-07-06T21:27:29.268300Z,0.90,NaN,5.4,1.16


In [69]:
import json

datos=[]
for i in range(6):
    file="../Figures/datosjson_"+str(i)
    with open(file, "r") as archivo:
        data = json.load(archivo)
    datos.append(data)
    

In [70]:
cov_matrices = []
for i in range(6):
    cov_matrices.append(np.array(datos[i]['Covariance']))

In [71]:
cov_matrices

[array([[23683434.08956666, -7482504.68269572],
        [-7482504.68269572, 23076439.22744324]]),
 array([[ 7693426.86133857, -6823952.50352896],
        [-6823952.50352896, 26516620.30147037]]),
 array([[44073456.563241  ,  -268518.05843598],
        [ -268518.05843598, 27670056.10147353]]),
 array([[ 8848368.20917511, -4026247.62006571],
        [-4026247.62006571, 25189221.12884306]]),
 array([[33042473.58570345,  3054639.48391083],
        [ 3054639.48391083, 13434698.28908999]]),
 array([[ 9723465.07889593, -7677083.07661694],
        [-7677083.07661694, 18175843.84263376]])]

In [75]:
convnresu='/Users/cecilia/CONVN/output/july_detections/from_stream/CI.CLC.2019-07-05.csv'

In [77]:
dfcon=pd.read_csv(convnresu)

In [97]:
p_times = df_f['p_arrival_time'].iloc[1]
p_time = UTCDateTime(p_times)

# Filtrar el DataFrame para encontrar el dato donde p_time está contenido dentro de start_time y end_time
datoc = dfcon[(dfcon['start_time'] <= p_time) & (dfcon['end_time'] >= p_time)]

# Imprimir el resultado

In [111]:
ws=datoc['clusters_prob']

In [116]:
wss=ws.tolist()[0]

In [118]:
wsss=eval(wss)

In [120]:
w0 = wsss[0]
w1 = wsss[1]
w2 = wsss[2]
w3 = wsss[3]
w4 = wsss[4]
w5 = wsss[5]

In [121]:
weights = [w0, w1, w2, w3, w4, w5] 

In [122]:
dato2=[]
for i in range(6):
    file="../Figures/datosjson_"+str(i)
    with open(file, "r") as archivo:
        data = json.load(archivo)
    dato2.append(data)
    

In [ ]:
dato2

In [132]:
mus = []
for i in range(6):
    mus.append(np.array(dato2[i]['Mean']))

In [133]:
mus

[array([-23041.24373151,  32044.59387201]),
 array([-7915.1580344 ,  7907.04237488]),
 array([15276.04154859, 17184.54034758]),
 array([  3835.58320631, -12156.52858867]),
 array([-25384.41677675, -50527.34361336]),
 array([ 13041.89905557, -20629.21364744])]

In [2]:


# Define the function S_P_t (Theoretical traveltime function) [SECONDS]
def S_P_t(x, y):
    st_loc = [1, 3]
    p_velocity = 7100   #[METERS/SECOND]
    s_velocity = 2900   #[METERS/SECOND]
    lent = (1 / s_velocity - 1 / p_velocity)
    dis = np.sqrt((x - st_loc[0]) ** 2 + (y - st_loc[1]) ** 2)
    sminp = dis * lent
    return sminp

# Define the Bayesian model
with pm.Model() as model:
    # Define the categories to choose the means
    category = pm.Categorical('category', p=weights)

    # Define the means corresponding to the categories
    mus = [pm.MvNormal(f'mu{i}', mu=mus[i], cov=cov_matrices[i], shape=2) for i in range(len(weights))]

    # Select the averages corresponding to the selected category.
    x = pm.Deterministic('x', pm.math.switch(
        pm.math.eq(category, 0), mus[0][0],
        pm.math.switch(pm.math.eq(category, 1), mus[1][0],
        pm.math.switch(pm.math.eq(category, 2), mus[2][0],
        pm.math.switch(pm.math.eq(category, 3), mus[3][0],
        pm.math.switch(pm.math.eq(category, 4), mus[4][0], mus[5][0]))))))
    
    y = pm.Deterministic('y', pm.math.switch(
        pm.math.eq(category, 0), mus[0][1],
        pm.math.switch(pm.math.eq(category, 1), mus[1][1],
        pm.math.switch(pm.math.eq(category, 2), mus[2][1],
        pm.math.switch(pm.math.eq(category, 3), mus[3][1],
        pm.math.switch(pm.math.eq(category, 4), mus[4][1], mus[5][1]))))))
        
    # Calculate t using the theoretical function
    t = S_P_t(x, y)

    # Likelihood of the observed data
    obs = pm.Normal('obs', mu=t, sigma=0.1, observed=t_observed)

with model:
    trace = pm.sample(300, tune=50, cores=1)

# Results summary
pm.summary(trace)

#pm.traceplot(trace)
#pm.autocorrplot(trace)

SyntaxError: unterminated string literal (detected at line 1) (3253628084.py, line 1)

## EJEMPLO EVENTO B

In [18]:
%%time
# Observed value of S-P is given by EQTransformer [SECONDS]
ts_observed = 33.118300
tp_observed = 31.718300
t_observed = ts_observed - tp_observed
print(f"The t_observed value is: {t_observed}")

# Define covariance matrix for each confidence ellipse calculated in 
# Building_Confidence_Ellipses_meters.ipynb (category) [METERS]
cov_matrices = [
    np.array([[23683275.01936196, -7482454.36868832], [-7482454.36868832, 23076283.80724357]]), 
    np.array([[7693375.19143287, -6823906.54539936], [-6823906.54539936, 26516441.70716951]]), 
    np.array([[44073160.78463332, -268516.2949894], [-268516.2949894, 27669869.60634939]]),
    np.array([[8848308.84080658, -4026220.50499397], [-4026220.50499397, 25189051.49760774]]), 
    np.array([[33042252.19084654, 3054618.90179884], [3054618.90179884, 13434607.86234287]]),
    np.array([[9723399.84428508, -7677031.46722373], [-7677031.46722373, 18175721.46713911]])
]

# Define weights for each ellipse given by ConvNetQuake (category) [DIMENSIONLESS]
w0 = 0.06656365
w1 = 0.13407934
w2 = 0.018142378
w3 = 0.2691841
w4 = 0.038651247
w5 = 0.2805622
weights = [w0, w1, w2, w3, w4, w5]  # Parameters varies on each event (need to be adjusted)

# Define specific bidimensional means for each ellipse calculated in 
# Building_Confidence_Ellipses_meters.ipynb (category) [METERS]
mus = [
    np.array([-23041.166265774566, 32044.48603132775]),
    np.array([-7915.131452346466, 7907.0157733109045]),
    np.array([15275.99036053178, 17184.482395381565]),
    np.array([3835.570302316145, -12156.487658598366]),
    np.array([-25384.331869022317, -50527.17341376274]),
    np.array([13041.855259334008, -20629.14421071338])
]

# Define the function S_P_t (Theoretical traveltime function) [SECONDS]
def S_P_t(x, y):
    st_loc = [1, 3]
    p_velocity = 7100   #[METERS/SECOND]
    s_velocity = 2900   #[METERS/SECOND]
    lent = (1 / s_velocity - 1 / p_velocity)
    dis = np.sqrt((x - st_loc[0]) ** 2 + (y - st_loc[1]) ** 2)
    sminp = dis * lent
    return sminp

# Define the Bayesian model
with pm.Model() as model:
    # Define the categories to choose the means
    category = pm.Categorical('category', p=weights)

    # Define the means corresponding to the categories
    mus = [pm.MvNormal(f'mu{i}', mu=mus[i], cov=cov_matrices[i], shape=2) for i in range(len(weights))]

    # Select the averages corresponding to the selected category.
    x = pm.Deterministic('x', pm.math.switch(
        pm.math.eq(category, 0), mus[0][0],
        pm.math.switch(pm.math.eq(category, 1), mus[1][0],
        pm.math.switch(pm.math.eq(category, 2), mus[2][0],
        pm.math.switch(pm.math.eq(category, 3), mus[3][0],
        pm.math.switch(pm.math.eq(category, 4), mus[4][0], mus[5][0]))))))
    
    y = pm.Deterministic('y', pm.math.switch(
        pm.math.eq(category, 0), mus[0][1],
        pm.math.switch(pm.math.eq(category, 1), mus[1][1],
        pm.math.switch(pm.math.eq(category, 2), mus[2][1],
        pm.math.switch(pm.math.eq(category, 3), mus[3][1],
        pm.math.switch(pm.math.eq(category, 4), mus[4][1], mus[5][1]))))))
    
    # Calculate t using the theoretical function
    t = S_P_t(x, y)

    # Likelihood of the observed data
    obs = pm.Normal('obs', mu=t, sigma=0.1, observed=t_observed)

with model:
    trace = pm.sample(3000, tune=500, cores=4)

# Results summary
pm.summary(trace)

The t_observed value is: 1.3999999999999986


/opt/miniforge3/envs/pymc3EQT/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [category]
>NUTS: [mu5, mu4, mu3, mu2, mu1, mu0]


Sampling 4 chains for 500 tune and 3_000 draw iterations (2_000 + 12_000 draws total) took 22 seconds.
/opt/miniforge3/envs/pymc3EQT/lib/python3.10/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
Got error No model on context stack. trying to find log_likelihood in translation.
/opt/miniforge3/envs/pymc3EQT/lib/python3.10/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


CPU times: user 6.58 s, sys: 277 ms, total: 6.85 s
Wall time: 26.6 s


/opt/miniforge3/envs/pymc3EQT/lib/python3.10/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
category,4.000,0.000,4.000,4.000,0.000,0.000,12000.0,12000.0,NaN
mu0[0],32222.373,4925.203,22773.292,41296.177,46.750,33.095,11121.0,8577.0,1.0
mu0[1],99852.049,4822.466,90996.179,108931.465,44.583,31.545,11691.0,9293.0,1.0
mu1[0],47279.414,2793.907,42083.916,52579.013,25.625,18.129,11884.0,10052.0,1.0
mu1[1],75811.214,5165.877,65870.485,85292.454,47.328,33.587,11917.0,9954.0,1.0
mu2[0],70508.981,6629.367,57881.582,82671.754,51.113,36.433,16808.0,8370.0,1.0
mu2[1],84970.872,5198.052,75412.672,94630.664,42.208,30.115,15146.0,9477.0,1.0
mu3[0],59102.746,2965.836,53773.855,64830.346,24.513,17.334,14634.0,9563.0,1.0
mu3[1],55604.523,5002.193,46198.429,64903.864,43.353,30.675,13330.0,9542.0,1.0
mu4[0],4529.634,1729.172,1139.400,7273.336,19.204,13.580,8187.0,7842.0,1.0


## EJEMPLO EVENTO C

In [19]:
%%time
# Observed value of S-P is given by EQTransformer [SECONDS]
ts_observed = 44.938300
tp_observed = 43.538300
t_observed = ts_observed - tp_observed
print(f"The t_observed value is: {t_observed}")

# Define covariance matrix for each confidence ellipse calculated in 
# Building_Confidence_Ellipses_meters.ipynb (category) [METERS]
cov_matrices = [
    np.array([[23683275.01936196, -7482454.36868832], [-7482454.36868832, 23076283.80724357]]), 
    np.array([[7693375.19143287, -6823906.54539936], [-6823906.54539936, 26516441.70716951]]), 
    np.array([[44073160.78463332, -268516.2949894], [-268516.2949894, 27669869.60634939]]),
    np.array([[8848308.84080658, -4026220.50499397], [-4026220.50499397, 25189051.49760774]]), 
    np.array([[33042252.19084654, 3054618.90179884], [3054618.90179884, 13434607.86234287]]),
    np.array([[9723399.84428508, -7677031.46722373], [-7677031.46722373, 18175721.46713911]])
]

# Define weights for each ellipse given by ConvNetQuake (category) [DIMENSIONLESS]
w0 = 0.0666608
w1 = 0.13491394
w2 = 0.018160287
w3 = 0.27093622
w4 = 0.03867901
w5 = 0.28238913
weights = [w0, w1, w2, w3, w4, w5]  # Parameters varies on each event (need to be adjusted)

# Define specific bidimensional means for each ellipse calculated in 
# Building_Confidence_Ellipses_meters.ipynb (category) [METERS]
mus = [
    np.array([-23041.166265774566, 32044.48603132775]),
    np.array([-7915.131452346466, 7907.0157733109045]),
    np.array([15275.99036053178, 17184.482395381565]),
    np.array([3835.570302316145, -12156.487658598366]),
    np.array([-25384.331869022317, -50527.17341376274]),
    np.array([13041.855259334008, -20629.14421071338])
]

# Define the function S_P_t (Theoretical traveltime function) [SECONDS]
def S_P_t(x, y):
    st_loc = [1, 3]
    p_velocity = 7100   #[METERS/SECOND]
    s_velocity = 2900   #[METERS/SECOND]
    lent = (1 / s_velocity - 1 / p_velocity)
    dis = np.sqrt((x - st_loc[0]) ** 2 + (y - st_loc[1]) ** 2)
    sminp = dis * lent
    return sminp

# Define the Bayesian model
with pm.Model() as model:
    # Define the categories to choose the means
    category = pm.Categorical('category', p=weights)

    # Define the means corresponding to the categories
    mus = [pm.MvNormal(f'mu{i}', mu=mus[i], cov=cov_matrices[i], shape=2) for i in range(len(weights))]

    # Select the averages corresponding to the selected category.
    x = pm.Deterministic('x', pm.math.switch(
        pm.math.eq(category, 0), mus[0][0],
        pm.math.switch(pm.math.eq(category, 1), mus[1][0],
        pm.math.switch(pm.math.eq(category, 2), mus[2][0],
        pm.math.switch(pm.math.eq(category, 3), mus[3][0],
        pm.math.switch(pm.math.eq(category, 4), mus[4][0], mus[5][0]))))))
    
    y = pm.Deterministic('y', pm.math.switch(
        pm.math.eq(category, 0), mus[0][1],
        pm.math.switch(pm.math.eq(category, 1), mus[1][1],
        pm.math.switch(pm.math.eq(category, 2), mus[2][1],
        pm.math.switch(pm.math.eq(category, 3), mus[3][1],
        pm.math.switch(pm.math.eq(category, 4), mus[4][1], mus[5][1]))))))
    
    # Calculate t using the theoretical function
    t = S_P_t(x, y)

    # Likelihood of the observed data
    obs = pm.Normal('obs', mu=t, sigma=0.1, observed=t_observed)

with model:
    trace = pm.sample(3000, tune=500, cores=4)

# Results summary
pm.summary(trace)

The t_observed value is: 1.3999999999999986


/opt/miniforge3/envs/pymc3EQT/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [category]
>NUTS: [mu5, mu4, mu3, mu2, mu1, mu0]


Sampling 4 chains for 500 tune and 3_000 draw iterations (2_000 + 12_000 draws total) took 21 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
Got error No model on context stack. trying to find log_likelihood in translation.
/opt/miniforge3/envs/pymc3EQT/lib/python3.10/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


CPU times: user 6.95 s, sys: 281 ms, total: 7.23 s
Wall time: 26.7 s


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
category,3.750,0.433,3.000,4.000,0.216,0.166,4.0,4.0,5.618624e+15
mu0[0],32276.145,4855.700,23043.032,41259.511,40.700,29.182,14242.0,9034.0,1.000000e+00
mu0[1],99832.031,4794.562,90899.983,108913.854,38.508,27.239,15492.0,9235.0,1.000000e+00
mu1[0],47345.907,2744.748,42022.543,52338.696,24.248,17.147,12797.0,9467.0,1.000000e+00
mu1[1],75705.589,5109.777,65739.616,85052.923,43.748,30.935,13634.0,9529.0,1.000000e+00
mu2[0],70484.993,6769.373,57528.207,82924.213,56.952,40.340,14131.0,8528.0,1.000000e+00
mu2[1],84949.745,5183.457,75675.601,95022.274,44.042,31.220,13814.0,8961.0,1.000000e+00
mu3[0],46312.571,22296.427,6829.677,63555.781,11064.847,8466.966,7.0,27.0,1.530000e+00
mu3[1],42684.943,23000.805,1818.006,62961.038,11304.986,8636.370,7.0,26.0,1.530000e+00
mu4[0],10818.649,11417.026,14.017,34634.232,5490.326,4179.127,7.0,26.0,1.530000e+00
